### Part 2: Periodicity of the motion.

This was NOT EASY. From the beginning I knew I had to look into the equations of motions to find some recurring pattern which simplified the dynamics. By looking at the extensive examples provided in the assignment I could tell that the motion on the various axes were decoupled and also that there were a lot of conserved quantities. In particular the sum of the components of both velocity and position of the 4 bodies on every axes were constant. In other words, if $X^i_\alpha(t)$ are the components of the position of body at time $t$ and $V^i_\alpha(t)$ are the ones of its velocity, it holds that:

\begin{equation}
\sum_{\alpha = 1}^{4} X^i_\alpha(t) = \sum_{\alpha = 1}^{ 4} X^i_\alpha(0)\\
    \sum_{\alpha = 1}^{4} V^i_\alpha(t) = \sum_{\alpha = 1}^{ 4} V^i_\alpha(0) = 0 \hspace{0.5 cm} \forall i = 1, 2, 3
\end{equation}

I thought I could use these to simplify the dynamics, but the fact that the dynamic of one bodies depends on all the others made this seem difficult.

Not having any useful insight (I thought I didn't) I gave up and looked at some hint on reddit, only to find that the decoupling of the motion on the axes was in fact the right path to follow. With this new certainty, I generalized my classes to work on variable dimension by specifying it in the constructors of the objects.

It then occurred to me. The decoupling essentially means that the 3 dimensional system is simply 3 copies of the same 1 dimensional system, all starting from a different initial condition. These means that if I I know the periodicity of the one dimensional system starting from the 3 initial conditions, the periodicity of the 3d one will be the ***least common multiplier between these 3 periodicities***. 

\begin{equation}
\hspace{0.5 cm} X - - - X - - - X - - - X \\
\hspace{0.6 cm}  X - X - X - X - X - X - X \\ 
\hspace{0.6 cm}X - - \hspace{0.2 cm}X - - \hspace{0.1 cm}X - - \hspace{0.1 cm}X - - \hspace{0.2 cm}X  
\end{equation}


In [201]:
import itertools as itools
import re
import numpy as np  
import math

    
class Moon:
    
    def __init__(self, dim, position = None, velocity = None, ):
        
        self.dimension = dim
        self.position = np.array(position).astype(int) if position is not None else np.zeros(dim).astype(int)
        self.velocity = np.array(velocity).astype(int) if velocity is not None else np.zeros(dim).astype(int)

        return

    
    def modulus_position(self):
        
        return sum([abs(x) for x in self.position])
    
    def modulus_velocity(self):
        
        return sum([abs(x) for x in self.velocity])


# Class of the Nbody problem. Init function needs a dictionary as input in the form
# bodies = {'NAME_OF_THE_BODY': OBJECT_OF_TYPE_<Moon>, ...}
class NbodySystem:
    
    def __init__(self, bodies = None):
        
        self.bodies = dict()
        self.dimension = 0
        if bodies is not None:
            self.bodies = bodies
            # Set dimension of system as the dimension of one item (the last in our implementation
            # but could be any) of the dictionary which is passed to the initializer
            self.dimension = self.bodies[list(self.bodies)[-1]].dimension
            
        return
    
    # Perform one integration step
    def integration_step(self):
            
        for pair in itools.combinations(list(self.bodies), 2):
            
            body1 = self.bodies[pair[0]]
            body2 = self.bodies[pair[1]]
            
            # Updating velocities
            for i in range(self.dimension):
                if body1.position[i] > body2.position[i]:
                    body1.velocity[i] -= 1
                    body2.velocity[i] += 1
                elif body1.position[i] < body2.position[i]:
                    body1.velocity[i] += 1
                    body2.velocity[i] -= 1
                    
        # Updating position AFTER the update of velocities
        for key in self.bodies:
            self.bodies[key].position += self.bodies[key].velocity
        

        return
    
    
    def total_energy(self):
        
        total = 0
        
        for key in self.bodies:
            potential = self.bodies[key].modulus_position()
            kinetic = self.bodies[key].modulus_velocity()
            total += potential*kinetic
            
        return total
    
    
    # Print the state of the system
    def print_system(self):
        
        for key in self.bodies:
            body = self.bodies[key]
            print(key, "<", body.position," > --- <", body.velocity, " >")
        
        print("Total energy: ", self.total_energy())
        print('\n')
        
        return

In [202]:
input_data = []
with open('input.txt', 'r') as infile:
    for line in infile:
        if line == '\n':
            continue
        else:
            found = re.findall('-?[0-9]*', line) # <--- Look at this bad boy, I'm so proud!
            input_data.append([int(x) for x in found if x is not ''])
            

In [203]:
# Names of the moons
moon_names = ['Io', 'Europa', 'Ganymede', 'Callisto']

# Introduce list of periods
period = [0, 0, 0]

# Cycle over the three dimensions
for i in range(3):

    # Declare moon dictionary 
    moon_dict = dict()

    # Get intial posisitons on current axis from input data
    init_positions = [[x[i]] for x in input_data]

    # create Moon dictionary with given initial positions
    for name, pos in zip(moon_names, init_positions):
        # Pass 1 as system is 1-dimensional
        moon_dict[name] = Moon(1, position = pos)

    # Instantiate Nbody system
    moon_system_1D = NbodySystem(moon_dict)

    # Boolean variable that check if the 4 bodies have returned to their previous position
    are_back = False
    
    # While the 4 bodies have not returned to their initial positions
    while not are_back:
        
        # Boolean variable for each body to see if it's back to its original position
        is_back = [False, False, False, False]
        
        # Integrate system for one step
        moon_system_1D.integration_step()
    
        # Check the system 
        for j, key in enumerate(moon_system_1D.bodies):
            
            # Useful not to write excessively long lines
            body = moon_system_1D.bodies[key]
            
            # If current body has returned to its intial position set boolean variable to True
            if [list(body.position), list(body.velocity)] == [init_positions[j], [0]]:
            
                is_back[j] = True
         
        # If all bodies are back, set global boolean variable to True
        are_back = all(is_back)
        
        # Increase counter
        period[i] += 1
    
print(period)

[231614, 116328, 102356]


In [204]:
# These are now the periods of the system on the three axis. We must find the least minimum multiple
# to determine the periodo of the global 3D system. With numpy, this is simply
print(np.lcm.reduce(period))

344724687853944
